In [103]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
data = pd.read_csv('stars.csv')

In [113]:
data.head()

,temperature_k,luminosityl-lo,radiusr-ro,absolute_magnitudemv,star_type,star_color,spectral_class
0,-0.779382,-0.598624,-0.459210,1.116745,0,Red,M
1,-0.782110,-0.598624,-0.459241,1.162414,0,Red,M
2,-0.828477,-0.598624,-0.459342,1.362213,0,Red,M
3,-0.807496,-0.598624,-0.459229,1.167171,0,Red,M
4,-0.897819,-0.598624,-0.459340,1.491607,0,Red,M


In [112]:
# data[['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv']]
# preprocessing.scale(data[['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv']])

data[['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv']] = preprocessing.StandardScaler().fit_transform(data[['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv']])

/Users/zhuochaozhong/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/zhuochaozhong/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [4]:
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('/', '-')

In [114]:
star_color = data['star_color'].unique()
spectral_class = data['spectral_class'].unique()

In [115]:
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

153 train examples
39 validation examples
48 test examples


In [116]:
def df_to_dataset(df, batch_size=32):
    
    df = df.copy()
    labels = df.pop('star_type')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))    
    ds = ds.batch(batch_size)
    return ds

In [117]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

In [118]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of star_color:', feature_batch['star_color'])
    print('A batch of star_type:', label_batch )

Every feature: ['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv', 'star_color', 'spectral_class']
A batch of star_color: tf.Tensor(
[b'Red' b'Red' b'Red' b'Red' b'Orange-Red' b'Red' b'Blue White'
 b'Blue-white' b'Blue white' b'Red' b'Red' b'Red' b'Red' b'Blue' b'White'
 b'Red' b'Blue' b'Blue' b'Red' b'Blue' b'Blue' b'yellow-white' b'Red'
 b'Red' b'Blue' b'Blue white' b'Red' b'Blue-white' b'Blue' b'Blue-white'
 b'Red' b'Blue-white'], shape=(32,), dtype=string)
A batch of star_type: tf.Tensor([1 1 5 5 3 5 2 3 2 0 0 1 5 2 2 0 3 4 0 2 4 3 5 1 4 2 1 3 5 3 1 3], shape=(32,), dtype=int64)


In [119]:
example = next(iter(train_ds))[0]
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example).numpy())

In [120]:
train_ds.element_spec

({'temperature_k': TensorSpec(shape=(None,), dtype=tf.float64, name=None),
  'luminosityl-lo': TensorSpec(shape=(None,), dtype=tf.float64, name=None),
  'radiusr-ro': TensorSpec(shape=(None,), dtype=tf.float64, name=None),
  'absolute_magnitudemv': TensorSpec(shape=(None,), dtype=tf.float64, name=None),
  'star_color': TensorSpec(shape=(None,), dtype=tf.string, name=None),
  'spectral_class': TensorSpec(shape=(None,), dtype=tf.string, name=None)},
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [121]:
feature_columns = []

star_color_col = feature_column.categorical_column_with_vocabulary_list(
    'star_color', star_color
)
one_hot_star_color = feature_column.indicator_column(star_color_col)
feature_columns.append(one_hot_star_color)

spectral_class_col = feature_column.categorical_column_with_vocabulary_list(
    'spectral_class', spectral_class
)
one_hot_spectral_class = feature_column.indicator_column(spectral_class_col)
feature_columns.append(one_hot_spectral_class)
    
for col in ['temperature_k', 'luminosityl-lo', 'radiusr-ro', 'absolute_magnitudemv']:
    feature_columns.append(feature_column.numeric_column(col))

In [122]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [123]:
feature_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='star_color', vocabulary_list=('Red', 'Blue White', 'White', 'Yellowish White', 'Blue white', 'Pale yellow orange', 'Blue', 'Blue-white', 'Whitish', 'yellow-white', 'Orange', 'White-Yellow', 'white', 'Blue ', 'yellowish', 'Yellowish', 'Orange-Red', 'Blue white ', 'Blue-White'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='spectral_class', vocabulary_list=('M', 'B', 'A', 'F', 'O', 'K', 'G'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 NumericColumn(key='temperature_k', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='luminosityl-lo', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='radiusr-ro', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='absolute_magnitudemv', shape=(1,), default_value=None, 

In [129]:
model.reset_states()
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [130]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [131]:
model.fit(train_ds, validation_data=val_ds, epochs=20)

Epoch 1/20
5/5 [==============================] - 0s 25ms/step - loss: 1.6592 - accuracy: 0.3464 - val_loss: 1.5114 - val_accuracy: 0.4103
Epoch 2/20
5/5 [==============================] - 0s 3ms/step - loss: 1.4702 - accuracy: 0.4902 - val_loss: 1.3248 - val_accuracy: 0.6667
Epoch 3/20
5/5 [==============================] - 0s 3ms/step - loss: 1.3078 - accuracy: 0.6209 - val_loss: 1.1553 - val_accuracy: 0.7436
Epoch 4/20
5/5 [==============================] - 0s 3ms/step - loss: 1.1581 - accuracy: 0.7320 - val_loss: 1.0013 - val_accuracy: 0.8974
Epoch 5/20
5/5 [==============================] - 0s 3ms/step - loss: 1.0208 - accuracy: 0.8627 - val_loss: 0.8618 - val_accuracy: 0.9487
Epoch 6/20
5/5 [==============================] - 0s 3ms/step - loss: 0.8939 - accuracy: 0.9150 - val_loss: 0.7384 - val_accuracy: 0.9744
Epoch 7/20
5/5 [==============================] - 0s 3ms/step - loss: 0.7799 - accuracy: 0.9346 - val_loss: 0.6284 - val_accuracy: 0.9744
Epoch 8/20
5/5 [=================